## Required parameters

| Parameter                         | Symbol     | Units       | Default Value (RGB) | **Description**                                              |
|---------------------------------------|----------------|-----------------|--------------------:|----------------------------------------------------------|
| Range                                 | $R$            | km              |                 400 | Distance from spectral source (Earth)                    |
| Center Wavelength                     | $\lambda$      | nm              |                 625 | Center wavelength in band                                |
| Aperture Diameter                     | $D_{ap}$       | mm              |                11.4 | Collecting area diameter                                 |
| Obscuration Diameter                  | $D_{obs}$      | mm              |                   0 | Diameter of obscuration (eg Cassegrain secondary mirror) |
| Detector Pixel X-Side Count (Smaller) | $n_x$          | -               |                2200 | Short side pixel count                                   |
| Detector Pixel Y-Side Count           | $n_y$          | -               |                3208 | Long side pixel count                                    |
| Detector Pixel Pitch                  | $D$            | um              |                   6 | Distance between pixels (square pixels assumed)          |
| Focal Length                          | $f$            | mm              |                  16 | Focal length of lens                                     |
| Bandwidth                             | $B$           | nm              |                 300 | Bandwidth of filter + pixel                              |
| Spectral Radiance in Band             | $L_{typ}$      | W/(sr\*m^2\*um) |               20.45 | Average spectral radiance of source                      |
| Quantum efficiency over BW            | $\eta$         | -               |                0.35 | Quantum efficiency of pixels                             |
| Exposure Time                         | $t$            | s               |              1.3E-4 | Total exposure time                                      |
| Well Capacity                         | $n_{ev_{max}}$ | e-              |               12000 | Maximum capacity of electron well                        |
| Dynamic Range                         | $DR$           | dB              |                  55 | Dynamic range of sensor                                  |

## Losses

| Loss source                        | Symbol     | Default Value |
|------------------------------------|------------|--------------:|
| Loss due to atmospheric absorption | $L_{abs}$  |          0.11 |
| Loss due to atmospheric scatter    | $L_{scat}$ |         0.058 |
| Loss due to optics                 | $L_{opt}$  |           0.2 |

## Parameter input and unit conversion

In [1]:
R = 400
lambda_c = 625
D_ap = 11.4
D_obs = 0
n_x = 2200
n_y = 3208
D = 6
f = 16
BW = 300
L_typ = 20.45
eta = 0.35
t = 1.3e-4
n_evmax = 12000
DR = 55

# Losses
L_abs = 0.11
L_scat = 0.0580
L_opt = 0.2
L_crop = 0.6

# Unit conversion

R *= 1e3
lambda_c *= 1e-9
D_ap *= 1e-3
D_obs *= 1e-3
D *= 1e-6
f *= 1e-3
BW *= 1e-9
L_typ *= 1e6


## Helper functions and imports

In [2]:
import numpy as np
import scipy as sp
import scipy.constants

def db2pow(db):
    return 10 ** (db/10)

def db2volt(db):
    return 10 ** (db/20)

def pow2db(power):
    return 10 * np.log10(power)

## Additional parameters and constants
| Parameter               | Symbol      | Units | Description                                       |
|-------------------------|-------------|------:|---------------------------------------------------|
| Aperture area           | $A_{ap}$    |  m\^2 | Area of aperture (assuming circular)              |
| Obscuration area        | $A_{obs}$   |  m\^2 | Area of obscuration (assuming circularity)        |
| Detector area           | $A_{det}$   |  m\^2 | Area of detector (assuming square pixels)         |
| Instantaneous pixel FOV | $iFOV_{p}$ |   deg | Angle subtended by pixel in combination with lens |
| Pixel smear             | $r_{smear}$ |     m | Ground distance moved during exposure time        |

In [3]:
# Constants

R_e = 6378.137 * 1e3 # earth radius
h = sp.constants.h
c = sp.constants.c

A_ap = np.pi * (D_ap ** 2) / 4
A_obs= np.pi * (D_obs ** 2) / 4
A_det = D**2 * n_x * n_y
iFOV_px = np.rad2deg(2 * np.arctan(0.5 * D / f)) # pixel iFOV
pix_smear = t * np.sqrt(398600000.5/(R+ R_e)) * 1000
# 

## Signal

In [5]:
P = A_det * ((A_ap - A_obs)/(f ** 2)) * BW * L_typ * (1-L_abs) * (1-L_scat) * (1 - L_opt) * (1-L_crop)
phi_det = (lambda_c * P) / (c * h)
phi = phi_det / (n_x * n_y)
n_ev = phi * eta * t
saturation = n_ev / n_evmax

# n_ev = total signal

## Noise

| Noise source        | Symbol    | Default value [e-] |
|---------------------|-----------|-------------------:|
| Shot noise          | $n_{sh}$  |    $\sqrt{n_{ev}}$ |
| Dark current noise  | $n_d$     |             $100t$ |
| Quantization noise  | $n_q$     |                $5$ |
| Fixed pattern noise | $n_{fpn}$ | $5e-4 * n_{ev}$    |
| Readout noise       | $n_r$     | $n_{ev_{max}}/DR$  |

In [6]:
n_sh = np.sqrt(n_ev)
n_d = 100 * t
n_q = 5
n_fpn = 0.0005 * n_ev
n_r = n_evmax /(db2volt(DR))

## SNR Calculation

In [7]:
snr = n_ev / np.sqrt(n_sh ** 2 + n_d ** 2 + n_q ** 2 + n_fpn ** 2 + (n_r ** 2) ** 2)

In [ ]:
# Results

